In [39]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import pandas as pd
import shutil
import random
import numpy as np

In [3]:
sys.path.insert(0, '..')
from function import *
from createimages import *

In [4]:
artists = pd.read_csv('../raw_data/artists.csv')

In [5]:
#drop id, years, nationality, bio, wikipedia
artists.drop(labels=['id','years','nationality','bio', 'wikipedia'], axis=1, inplace = True)
#rename 'Albrecht Dürer' to 'Albrecht Durer'
artists['name'] = artists['name'].replace(to_replace='Albrecht D.rer', value = 'Albrecht Durer', regex = True)
artists['name'] = artists['name'].str.replace(' ', '_')
artists.head()

,name,genre,paintings
0,Amedeo_Modigliani,Expressionism,193
1,Vasiliy_Kandinskiy,"Expressionism,Abstractionism",88
2,Diego_Rivera,"Social Realism,Muralism",70
3,Claude_Monet,Impressionism,73
4,Rene_Magritte,"Surrealism,Impressionism",194


In [6]:
#create data frame with artist name, genre, filename, filepath
path = r'..\raw_images'
#get image information
img_info = get_image_info(path)
#convert lists to dataframe
zipped = zip(['name', 'filename', 'filepath'], img_info)
artlabels = pd.DataFrame(dict(list(zipped)))

In [7]:
#extract the artist name and genre
genre = artists.loc[:,['name', 'genre']]
#label each file with their respective genres
artlabels = artlabels.merge(genre, how='left', on='name')
artlabels.head()

,name,filename,filepath,genre
0,Albrecht_Durer,Albrecht_Durer_(1).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...,Northern Renaissance
1,Albrecht_Durer,Albrecht_Durer_(10).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...,Northern Renaissance
2,Albrecht_Durer,Albrecht_Durer_(100).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...,Northern Renaissance
3,Albrecht_Durer,Albrecht_Durer_(101).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...,Northern Renaissance
4,Albrecht_Durer,Albrecht_Durer_(102).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...,Northern Renaissance


In [8]:
#separate dataframe into train and test sets
genres = artlabels.genre.unique()
train_set = []

for i in genres:
    df = artlabels.loc[artlabels.genre == str(i)]
    df.reset_index(inplace=True, drop=True)
    train_size = 0.8
    train_index = round(len(df)*train_size)
    for x in range(train_index):
        train_set.append(list(df.iloc[x]))


In [9]:
train_set=pd.DataFrame(train_set, columns=['name', 'filename', 'filepath', 'genre'])
test_set=artlabels[~artlabels.filename.isin(list(train_set.filename))]
test_set.reset_index(inplace=True, drop=True)

In [10]:
#create a new folder for the csv files
datapath = '..\data'
create_new_folder(datapath)

..\data path already exists


In [8]:
'''#create a new csv file with at most one genre for an artist
newfile = 'artists_clean.csv'
fullpath = os.path.join(datapath, newfile)
artists.to_csv(fullpath)'''

In [9]:
'''#create a csv file from the new dataframe
newfile = 'artlabels.csv'
fullpath = os.path.join(datapath, newfile)
artlabels.to_csv(fullpath)'''

In [11]:
rawpath = r'..\raw_images'
trainpath = r'..\train_images'
testpath = r'..\test_images'

In [16]:
[create_new_folder(i) for i in [rawpath, trainpath, testpath]]

..\raw_images path already exists
..\train_images created
..\test_images created


[None, None, None]

In [18]:
for i in genres:
    train = os.path.join(trainpath, i)
    test = os.path.join(testpath, i)
    create_new_folder(train)
    create_new_folder(test)

..\train_images\Northern Renaissance created
..\test_images\Northern Renaissance created
..\train_images\Impressionism created
..\test_images\Impressionism created
..\train_images\Expressionism created
..\test_images\Expressionism created
..\train_images\Byzantine Art created
..\test_images\Byzantine Art created
..\train_images\Pop Art created
..\test_images\Pop Art created
..\train_images\Impressionism,Post-Impressionism created
..\test_images\Impressionism,Post-Impressionism created
..\train_images\Baroque created
..\test_images\Baroque created
..\train_images\Social Realism,Muralism created
..\test_images\Social Realism,Muralism created
..\train_images\Realism,Impressionism created
..\test_images\Realism,Impressionism created
..\train_images\Symbolism,Expressionism created
..\test_images\Symbolism,Expressionism created
..\train_images\Mannerism created
..\test_images\Mannerism created
..\train_images\Romanticism created
..\test_images\Romanticism created
..\train_images\Primitivism,

In [21]:
for i in range(len(train_set)):
    shutil.copy(train_set.filepath[i], os.path.join(trainpath, train_set.genre[i]))
for i in range(len(test_set)):
    shutil.copy(test_set.filepath[i],os.path.join(testpath, test_set.genre[i]))
    

In [13]:
sample_df = artists.loc[(artists.genre == 'Impressionism') | (artists.genre == 'Pop Art') | (artists.genre == 'Abstract Expressionism')]
sample_df.reset_index(inplace=True, drop=True)

In [14]:
#clear old images to procure a new sample
num_skipped = clear_old_images(trainpath, sample_df)

Deleted 0 images


In [14]:
#extract sample images
image_list = get_random_imgs(rawpath, imagepath, sample_df, 0.5)

..\images\Impressionism path already exists
..\images\Post-Impressionism path already exists
..\images\Northern Renaissance path already exists
..\images\Impressionism path already exists
..\images\Northern Renaissance path already exists
..\images\Impressionism path already exists
..\images\Northern Renaissance path already exists
..\images\Northern Renaissance path already exists
..\images\Impressionism path already exists
..\images\Post-Impressionism path already exists
..\images\Post-Impressionism path already exists
..\images\Post-Impressionism path already exists
Generated 1548 new images


In [22]:
df_labels = pd.DataFrame(columns=['label', 'filename', 'filepath'])
for i in range(len(image_list)):
    df_labels.loc[i] = image_list[i]  

In [23]:
df_labels.head()

,label,filename,filepath
0,Impressionism,Claude_Monet_3.jpg,..\images\Impressionism\Claude_Monet_3.jpg
1,Impressionism,Claude_Monet_63.jpg,..\images\Impressionism\Claude_Monet_63.jpg
2,Impressionism,Claude_Monet_68.jpg,..\images\Impressionism\Claude_Monet_68.jpg
3,Impressionism,Claude_Monet_58.jpg,..\images\Impressionism\Claude_Monet_58.jpg
4,Impressionism,Claude_Monet_48.jpg,..\images\Impressionism\Claude_Monet_48.jpg


In [24]:
df_labels.to_csv(r'..\data\sample.csv')

In [25]:
df_labels_list = list(df_labels.filename)

In [39]:
not_sampled = artlabels[~artlabels.filename.isin(df_labels_list)]
not_sampled = not_sampled.loc[(not_sampled.genre == 'Impressionism') | (not_sampled.genre == 'Pop Art') | (not_sampled.genre == 'Abstract Expressionism')]
not_sampled.reset_index(inplace=True)
not_sampled.drop(labels='index', axis=1, inplace=True)

In [40]:
not_sampled

,name,filename,filepath,genre
0,Albrecht_Durer,Albrecht_Durer_(10).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...,Northern Renaissance
1,Albrecht_Durer,Albrecht_Durer_(104).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...,Northern Renaissance
2,Albrecht_Durer,Albrecht_Durer_(106).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...,Northern Renaissance
3,Albrecht_Durer,Albrecht_Durer_(108).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...,Northern Renaissance
4,Albrecht_Durer,Albrecht_Durer_(110).jpg,..\raw_images\Albrecht_Durer\Albrecht_Durer_(1...,Northern Renaissance


In [ ]:
testpath = r'..\test_images'
create_new_folder(testpath)

In [ ]:
num_skipped = clear_old_images(testpath, not_sampled)

In [ ]:
#this would be 25% of the total images for each class from the images not sampled
test_list = get_random_imgs(rawpath, testpath, not_sampled, 0.5)

In [ ]:
df_labels = pd.DataFrame(columns=['label', 'filename', 'filepath'])
for i in range(len(image_list)):
    df_labels.loc[i] = image_list[i]

In [85]:
empty_dataframe = pd.DataFrame()
list_dataframe = pd.DataFrame([[1, 2, 3], [1, 2,3]])
print(list_dataframe)
list_dataframe = np.transpose(list_dataframe)
result_dataframe = empty_dataframe.append(list_dataframe)
print(result_dataframe)

   0  1  2
0  1  2  3
1  1  2  3
   0  1
0  1  1
1  2  2
2  3  3
